<div align='center'><font size="5" color='#353B47'>Turn your time series into images</font></div>
<div align='center'><font size="4" color="#353B47">Different ways to leverage your inner glubibulga</font></div>
<br>
<hr>

# <div id="summary">Table of Contents: GAF</div>
**<font size="2"><a href="#chap1">1. On multiple time series</a></font>**
**<br><font size="2"><a href="#chap2">2. On Optiver data</a></font>**

In [ ]:
# Libs
!pip install pyts

from mpl_toolkits.axes_grid1 import ImageGrid
from pyts.image import GramianAngularField
from pyts.datasets import load_gunpoint
import pandas as pd
import numpy as np

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import plotly.express as px
import glob

**<font color="blue" size="6">GAF : Grammian Angular Fields</font>**

## <div id="chap1">1. On multiple time series (quickwin)</div>

In [ ]:
def generate_time_series(n, length = 900):
    
    """
    Generate n time series of size length

    Parameters
    ----------
    n: int
        number of time series to generate
    length: int
        size of the time series
    """
    
    time_series = []
    
    for _ in range(n):
        time_serie = pd.Series(
            np.random.randn(length), 
            index = np.arange(length)
        ).cumsum()
        time_series.append(time_serie)
    return time_series

ts = generate_time_series(n = 50)

In [ ]:
def plot_50_encoded_ts(time_series, method = 'summation'):

    """
    Encode and plot 50 time series

    Parameters
    ----------
    time_series: list
        list of time series to encode
    method: str
        summation or difference GAF
    """
    
    if method == 'summation':
        gasf = GramianAngularField(image_size=30, method='summation')
        X = gasf.fit_transform(time_series)

    elif method == 'difference':
        gadf = GramianAngularField(image_size=30, method='difference')
        X = gadf.fit_transform(time_series)

    fig = make_subplots(
        rows=5, cols=10, 
        #subplot_titles=("GASF", "GADF"),
        vertical_spacing = 0.1
    )

    for index in range(50):

        fig.add_trace(
            go.Heatmap(
                z=X[index]
            ),
            row = index//10+1, col = index%10+1,
        )


    fig.update_traces(showscale=False)
    fig.update_layout(
        showlegend = False,
        template = 'plotly_dark',
        width = 1200,
        height = 900

    )

    fig.update_yaxes(title = '', visible = True, showticklabels = False)    
    fig.update_xaxes(title = '', visible = True, showticklabels = False)

    fig.show()
    
plot_50_encoded_ts(ts)

--------

**<font size="2"><a href="#summary">Back to summary</a></font>**

## <div id="chap2">2. On Optiver data, per stock_id-time_id</div>

In [ ]:
# helpers
def get_snapshot_data(stock_id, time_id):
    
    """
    Import and filter Optiver data

    Parameters
    ----------
    stock_id: int
    time_id : int
    """
    
    book_table = pq.read_table(f'../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id={stock_id}').to_pandas()
    trade_table =  pq.read_table(f'../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id={stock_id}').to_pandas()
    
    book_table = book_table[book_table['time_id']==time_id]
    book_table.loc[:,'stock_id'] = str(stock_id)
    
    trade_table = trade_table[trade_table['time_id']==time_id]
    trade_table.loc[:,'stock_id'] = str(stock_id)
    
    return book_table, trade_table

def compute_wap(book_table):
    
    """
    Compute WAP and WAP_2

    Parameters
    ----------
    book_table: pd.DataFrame
    """
    
    book_table['WAP'] = (book_table['bid_price1'] * book_table['ask_size1'] + book_table['ask_price1'] * book_table['bid_size1']) / (book_table['bid_size1'] + book_table['ask_size1'])
    book_table['WAP_2'] = (book_table['bid_price2'] * book_table['ask_size2'] + book_table['ask_price2'] * book_table['bid_size2']) / (book_table['bid_size2'] + book_table['ask_size2'])

    return book_table

def plot_wap(book_table, second = False):
    
    """
    Plot time serie representing WAP or WAP_2

    Parameters
    ----------
    book_table: pd.DataFrame
    second: boolean
        whether to plot WAP_2 or not
    """
    
    stock_id = book_table.stock_id[0]
    time_id = book_table.time_id[0]
    
    if second:
        fig = px.line(book_table, 
                      x="seconds_in_bucket", 
                      y="WAP", 
                      title=f'WAP of stock_id_{stock_id}, time_id_{time_id}')
        
    else:
        fig = px.line(book_table, 
                      x="seconds_in_bucket", 
                      y="WAP_2", 
                      title=f'WAP of stock_id_{stock_id}, time_id_{time_id}')
        
    fig.show()

In [ ]:
def preprocess_optiver(stock_ids, time_ids):
    
    """
    Return a list of time series to be encoded

    Parameters
    ----------
    stock_ids: list
        list of stock ids 
    time_ids: list
        list of time ids
    """
    
    time_series = []
    
    for stock_id, time_id in zip(stock_ids, time_ids):
        
        # Import, filter and create WAP
        book_table, trade_table = get_snapshot_data(stock_id, time_id)
        book_table_with_wap = compute_wap(book_table)
        
        # Transform WAP as a time serie and append to the list of time series
        ts = pd.Series(book_table_with_wap['WAP'])
        time_series.append(ts)
        
    return time_series

time_series = preprocess_optiver(stock_ids = [0], time_ids = [5])

In [ ]:
def get_GAFs(time_series):

    """
    Encode time series list under GADF and GASF representation

    Parameters
    ----------
    time_series: list
        list of time series
    """
    
    gasf = GramianAngularField(image_size=33, method='summation')
    gadf = GramianAngularField(image_size=33, method='difference')
    
    X_gasf = gasf.fit_transform(time_series)
    X_gadf = gadf.fit_transform(time_series)
    
    return X_gasf, X_gadf

X_gasf, X_gadf = get_GAFs(time_series)

In [ ]:
def plot_GAF_plotly(GASF, GADF, save = True):
    
    """
    Plot GASF and GADF

    Parameters
    ----------
    GASF: list[list]
        GASF matrix
    GADF: list[list]
        GADF matrix
    save: boolean
        whether to save GAF plot or not
    """
    
    fig = make_subplots(
        rows=1, cols=2, 
        subplot_titles=("GASF", "GADF"),
        vertical_spacing = 0.1
    )

    fig.add_trace(
        go.Heatmap(
            z=GASF[0]
        ),
        row=1, col=1,
    )

    fig.add_trace(
        go.Heatmap(
            z=GADF[0],
        ),
        row=1, col=2
    )

    fig.update_traces(showscale=False)
    fig.update_layout(
        showlegend = False,
        template = 'plotly_dark',
        width = 1050,
        height = 600

    )

    fig.update_yaxes(title = '', visible = True, showticklabels = False)    
    fig.update_xaxes(title = '', visible = True, showticklabels = False)

    fig.show()

    if save:
        pass
#         fig.write_image("images/GAF_plotly.png")
#         fig.write_image("image_vectors/GAF_plotly.svg")

    
plot_GAF_plotly(X_gasf, X_gadf)

<hr>
<div align='justify'><font color="#353B47" size="4">Thank you for taking the time to read this notebook. I hope that I was able to answer your questions or your curiosity and that it was quite understandable. <u>any constructive comments are welcome</u>. They help me progress and motivate me to share better quality content. I am above all a passionate person who tries to advance my knowledge but also that of others. If you liked it, feel free to <u>upvote and share my work.</u> </font></div>
<br>
<div align='center'><font color="#353B47" size="3">Thank you and may passion guide you.</font></div>